# CONFLUENCE Tutorial - 5: Elevation-Based HRU Discretization (Bow River at Banff)

## Introduction

This tutorial demonstrates the most spatially detailed modeling approach in our series: elevation-based HRU discretization. Building on the semi-distributed model from Tutorial 4, we now further subdivide each GRU into multiple Hydrologic Response Units (HRUs) based on elevation bands. This approach captures the strong elevation controls on mountain hydrology while maintaining computational tractability.

### What is Elevation-Based HRU Discretization?

Elevation-based discretization creates multiple HRUs within each GRU by dividing the elevation range into bands:

- **Within-GRU heterogeneity**: Each GRU is subdivided into elevation bands (e.g., 200m intervals)
- **Process differentiation**: Different elevation bands experience different climate conditions and hydrological processes
- **Spatial detail**: Maximum spatial resolution while maintaining computational efficiency
- **Elevation controls**: Explicitly represents elevation-dependent processes like temperature lapse rates and snow dynamics

### Key Concepts

**Elevation Bands**: Altitude ranges (e.g., 1400-1600m, 1600-1800m) that group areas with similar elevation-dependent characteristics.

**HRU (Hydrologic Response Unit)**: The smallest modeling unit, representing areas with similar hydrological response characteristics - in this case, defined by both location (GRU) and elevation band.

**Elevation Band Size**: The vertical interval used to create bands (e.g., 200m), which controls the level of elevation detail captured.

**Minimum HRU Size**: Areas smaller than this threshold are merged with adjacent elevation bands to maintain computational efficiency.

### Why Elevation-Based Discretization?

In mountain watersheds like the Bow River, elevation is a primary control on:

1. **Temperature gradients**: Systematic temperature decrease with elevation affects snowmelt timing and evapotranspiration
2. **Precipitation patterns**: Orographic effects create elevation-dependent precipitation gradients
3. **Snow dynamics**: Accumulation, persistence, and ablation vary strongly with elevation
4. **Vegetation zones**: Different plant communities and growing seasons across elevation gradients
5. **Seasonal timing**: Earlier snowmelt at lower elevations, later at higher elevations

### Learning Outcomes

This tutorial will teach you how to:

1. **Apply elevation-based discretization** to existing GRU boundaries
2. **Configure elevation parameters** (band size, minimum HRU size)
3. **Manage increased model complexity** with many HRUs
4. **Interpret elevation-dependent results** across different zones
5. **Understand computational trade-offs** between detail and efficiency
6. **Analyze spatial patterns** in model outputs

## Step 1: Elevation-Based Setup with Data Reuse
Building on the semi-distributed modeling from Tutorial 02b, we now advance to the most spatially detailed approach: elevation-based HRU discretization. This represents the pinnacle of spatial complexity in our modeling series, subdividing each GRU into multiple elevation bands to capture the strong altitudinal controls on mountain hydrology while maintaining computational tractability.

### Modeling Evolution: Semi-Distributed → Elevation-Based HRUs

- **Spatial Units**: Multiple GRUs → Multiple HRUs within each GRU based on elevation bands
- **Discretization Method**: `GRUs` → `elevation` with configurable band sizes
- **Process Representation**: Sub-basin averaged → Elevation-dependent processes within sub-basins
- **Hydrological Detail**: Stream network routing → Stream network + elevation gradient effects
- **Scientific Focus**: Spatial connectivity → Elevation controls on snow, temperature, and timing

The same CONFLUENCE framework seamlessly handles this complexity increase while intelligent data reuse from Tutorial 02b eliminates redundant preprocessing, demonstrating efficient workflow management for the most detailed spatial modeling approach.


In [ ]:
# =============================================================================
# STEP 1: ELEVATION-BASED SETUP WITH DATA REUSE
# =============================================================================

# Import required libraries
import sys
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime
import xarray as xr
import numpy as np
import shutil

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Import main CONFLUENCE class
from CONFLUENCE import CONFLUENCE

# Set up plotting style
plt.style.use('default')
%matplotlib inline

# =============================================================================
# CONFIGURATION FOR ELEVATION-BASED BOW RIVER MODELING
# =============================================================================

print("\n🏔️  Configuring Elevation-Based Bow River Watershed...")

# Set directory paths
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/Users/darrieythorsson/compHydro/data/CONFLUENCE_data')  # ← Update this path

# Load semi-distributed configuration from Tutorial 02b and customize for elevation-based modeling
semi_distributed_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_semi_distributed.yaml'

if not semi_distributed_config_path.exists():
    print("⚠️  Tutorial 02b configuration not found. Loading template configuration...")
    config_template_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_template.yaml'
    with open(config_template_path, 'r') as f:
        config_dict = yaml.safe_load(f)
else:
    with open(semi_distributed_config_path, 'r') as f:
        config_dict = yaml.safe_load(f)

# Update for elevation-based discretization modeling
config_updates = {
    'CONFLUENCE_CODE_DIR': str(CONFLUENCE_CODE_DIR),
    'CONFLUENCE_DATA_DIR': str(CONFLUENCE_DATA_DIR),
    'DOMAIN_NAME': 'Bow_at_Banff_elevation',
    'EXPERIMENT_ID': 'elevation_tutorial',
    'POUR_POINT_COORDS': '51.1722/-115.5717',  # Same pour point as previous tutorials
    'DOMAIN_DEFINITION_METHOD': 'delineate',    # Reuse GRUs from semi-distributed
    'DOMAIN_DISCRETIZATION': 'elevation',       # KEY CHANGE: elevation-based HRUs
    'ELEVATION_BAND_SIZE': 200,                 # 200m elevation bands
    'MIN_HRU_SIZE': 4,                          # 4 km² minimum HRU size
    'SPATIAL_MODE': 'Distributed',              # Full spatial distribution
    'HYDROLOGICAL_MODEL': 'SUMMA',
    'ROUTING_MODEL': 'mizuRoute',               # Essential for multi-HRU routing
    'EXPERIMENT_TIME_START': '2011-01-01 01:00',
    'EXPERIMENT_TIME_END': '2018-12-31 23:00',
    'CALIBRATION_PERIOD': '2011-01-01, 2015-12-31',
    'EVALUATION_PERIOD': '2016-01-01, 2018-12-31',
    'SPINUP_PERIOD': '2011-01-01, 2011-12-31',
    'STATION_ID': '05BB001',
    'DOWNLOAD_WSC_DATA': True
}

config_dict.update(config_updates)

# Add experiment metadata
config_dict['NOTEBOOK_CREATION_TIME'] = datetime.now().isoformat()
config_dict['NOTEBOOK_CREATOR'] = 'CONFLUENCE_Tutorial_02c'
config_dict['SPATIAL_EVOLUTION'] = 'Semi-distributed to elevation-based HRU discretization'

# Save configuration
elevation_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_elevation.yaml'
with open(elevation_config_path, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False, sort_keys=False)

print(f"✅ Elevation-based configuration saved: {elevation_config_path}")

# =============================================================================
# INTELLIGENT DATA REUSE FROM TUTORIAL 02B
# =============================================================================

print(f"\n📂 Smart Data Reuse from Tutorial 02b...")

# Check for existing data from semi-distributed model tutorial
semi_distributed_domain = 'Bow_at_Banff_distributed'  # From Tutorial 02b
semi_distributed_data_dir = CONFLUENCE_DATA_DIR / f'domain_{semi_distributed_domain}'

if semi_distributed_data_dir.exists():
    print(f"✅ Found existing data from Tutorial 02b: {semi_distributed_data_dir}")
    
    # Define reusable data categories - focus on GRU boundaries and processed data
    reusable_data = {
        'GRU Boundaries': semi_distributed_data_dir / 'shapefiles' / 'river_basins',
        'River Network': semi_distributed_data_dir / 'shapefiles' / 'river_network',
        'Elevation (DEM)': semi_distributed_data_dir / 'attributes' / 'elevation',
        'Soil Data': semi_distributed_data_dir / 'attributes' / 'soilclass', 
        'Land Cover': semi_distributed_data_dir / 'attributes' / 'landclass',
        'ERA5 Forcing': semi_distributed_data_dir / 'forcing' / 'raw_data',
        'WSC Observations': semi_distributed_data_dir / 'observations' / 'streamflow'
    }
    
    # Initialize CONFLUENCE first to create directory structure
    confluence = CONFLUENCE(elevation_config_path)
    project_dir = confluence.managers['project'].setup_project()
    
    def copy_with_name_adaptation(src_path, dst_path, old_name, new_name):
        """Copy files with name adaptation for new domain"""
        if not src_path.exists():
            return False
            
        dst_path.parent.mkdir(parents=True, exist_ok=True)
        
        if src_path.is_dir():
            # Copy directory contents with name adaptation
            for src_file in src_path.rglob('*'):
                if src_file.is_file():
                    rel_path = src_file.relative_to(src_path)
                    # Adapt filename
                    new_filename = src_file.name.replace(old_name, new_name)
                    dst_file = dst_path / rel_path.parent / new_filename
                    dst_file.parent.mkdir(parents=True, exist_ok=True)
                    shutil.copy2(src_file, dst_file)
            return True
        elif src_path.is_file():
            # Copy single file with name adaptation
            new_filename = dst_path.name.replace(old_name, new_name)
            dst_file = dst_path.parent / new_filename
            dst_file.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(src_path, dst_file)
            return True
        return False
    
    # Check availability and copy reusable data
    print(f"\n🔄 Copying and Adapting Reusable Data...")
    
    # Copy reusable data with appropriate naming
    for data_type, src_path in reusable_data.items():
        if src_path.exists():
            # Determine destination path
            rel_path = src_path.relative_to(semi_distributed_data_dir)
            dst_path = project_dir / rel_path
            
            # Copy with name adaptation
            success = copy_with_name_adaptation(
                src_path, dst_path, 
                semi_distributed_domain, config_dict['DOMAIN_NAME']
            )
            
            if success:
                print(f"   ✅ {data_type}: Copied and adapted")
            else:
                print(f"   ⚠️  {data_type}: Copy failed")
        else:
            print(f"   📋 {data_type}: Not found, will acquire fresh")
    
    print(f"\n💡 Elevation-Based Modeling Advantages:")
    elevation_benefits = [
        "Captures elevation-dependent temperature gradients and lapse rates",
        "Represents snowpack distribution and ablation timing across elevations",
        "Enables analysis of climate change impacts on different elevation zones",
        "Provides detailed spatial attribution of hydrological processes",
        "Maintains computational efficiency with smart HRU aggregation",
        "Builds on validated GRU boundaries from semi-distributed setup"
    ]
    
    for benefit in elevation_benefits:
        print(f"   🏔️  {benefit}")

    print(f"\n🔧 Elevation Discretization Configuration:")
    print(f"   📏 Elevation band size: {config_dict['ELEVATION_BAND_SIZE']} m")
    print(f"   📐 Minimum HRU size: {config_dict['MIN_HRU_SIZE']} km²")
    print(f"   🗺️  Base GRU structure: Reused from Tutorial 02b")
    print(f"   ⚙️  Processing: Multiple HRUs per GRU based on elevation bands")

else:
    print(f"⚠️  No existing data found from Tutorial 02b")
    print(f"   Tutorial 02b must be completed first for GRU boundaries")
    print(f"   Will initialize fresh project structure but require GRU setup")
    
    # Initialize CONFLUENCE and create project structure
    confluence = CONFLUENCE(elevation_config_path)
    project_dir = confluence.managers['project'].setup_project()

# Create pour point
pour_point_path = confluence.managers['project'].create_pour_point()

print(f"\n🏔️  Elevation-Based HRU Setup Summary:")
print(f"   📁 Project directory: {project_dir}")
print(f"   📍 Pour point: {config_dict['POUR_POINT_COORDS']}")
print(f"   🌊 Domain name: {config_dict['DOMAIN_NAME']}")
print(f"   🎯 Experiment ID: {config_dict['EXPERIMENT_ID']}")
print(f"   📊 Discretization: {config_dict['DOMAIN_DISCRETIZATION']} (elevation bands)")
print(f"   ⏱️  Simulation period: {config_dict['EXPERIMENT_TIME_START']} to {config_dict['EXPERIMENT_TIME_END']}")

print(f"\n✅ Section 1 Complete: Ready for elevation-based domain discretization")

## Step 2: Elevation-Based Discretization within Existing GRUs
The transition from semi-distributed to elevation-based modeling represents a sophisticated spatial refinement process. Rather than creating new watershed boundaries, we now subdivide the validated GRU structure from Tutorial 02b into elevation bands, transforming connected sub-basins into elevation-stratified hydrological response units that capture altitudinal controls on mountain hydrology.

### Scientific Context: Elevation-Based Spatial Analysis

**Elevation Band Principles:**
- **Altitudinal Stratification**: Division of each GRU into systematic elevation intervals
- **Process Differentiation**: Different elevation bands experience distinct climate conditions
- **Thermal Gradients**: Temperature lapse rates create elevation-dependent energy balances
- **Snow Zone Dynamics**: Accumulation and ablation timing varies systematically with elevation
- **Computational Efficiency**: Maintains manageable complexity while capturing key elevation effects

The elevation band size parameter critically controls model detail: smaller bands capture finer elevation gradients but increase computational demands, while larger bands reduce detail but maintain efficiency. The minimum HRU size prevents creation of computationally-inefficient micro-units.


In [ ]:

# =============================================================================
# STEP 2: ELEVATION-BASED DISCRETIZATION WITHIN EXISTING GRUS
# =============================================================================

print("=== Step 2: Elevation-Based Discretization within Existing GRUs ===")
print("Transforming GRU sub-basins into elevation-stratified hydrological response units")

# =============================================================================
# VERIFY EXISTING GRU STRUCTURE FROM TUTORIAL 02B
# =============================================================================

print(f"\n🗺️  Verifying GRU Structure from Tutorial 02b...")

# Check for existing GRU boundaries
basin_path = project_dir / 'shapefiles' / 'river_basins'
network_path = project_dir / 'shapefiles' / 'river_network'

basin_files = list(basin_path.glob('*.shp')) if basin_path.exists() else []
network_files = list(network_path.glob('*.shp')) if network_path.exists() else []

if basin_files and network_files:
    # Load existing GRU structure
    basins_gdf = gpd.read_file(basin_files[0])
    network_gdf = gpd.read_file(network_files[0])
    
    print(f"✅ GRU structure loaded from Tutorial 02b")
    print(f"   Base GRUs: {len(basins_gdf)} sub-basins")
    print(f"   Stream segments: {len(network_gdf)}")
    print(f"   Total area: {basins_gdf.geometry.area.sum() / 1e6:.1f} km²")
    
    # Analyze elevation characteristics if available
    if 'elevation' in basins_gdf.columns:
        print(f"   Elevation range: {basins_gdf['elevation'].min():.0f}m to {basins_gdf['elevation'].max():.0f}m")
        elevation_span = basins_gdf['elevation'].max() - basins_gdf['elevation'].min()
        expected_bands = int(elevation_span / config_dict['ELEVATION_BAND_SIZE']) + 1
        print(f"   Expected elevation bands: ~{expected_bands} bands ({config_dict['ELEVATION_BAND_SIZE']}m intervals)")
    
else:
    print(f"⚠️  GRU structure not found from Tutorial 02b")
    print(f"   Tutorial 02b must be completed first")
    print(f"   Elevation discretization requires existing GRU boundaries")
    raise FileNotFoundError("GRU boundaries required from Tutorial 02b")

# =============================================================================
# ELEVATION-BASED DISCRETIZATION PROCESS
# =============================================================================

print(f"\n🏔️  Elevation-Based Discretization Process...")
print(f"   Method: {config_dict['DOMAIN_DISCRETIZATION']} (elevation band subdivision)")
print(f"   Elevation band size: {config_dict['ELEVATION_BAND_SIZE']} m")
print(f"   Minimum HRU size: {config_dict['MIN_HRU_SIZE']} km²")
print(f"   Base GRU structure: Inherited from semi-distributed setup")

print(f"\n🔧 Elevation Discretization Workflow:")
discretization_steps = [
    "GRU boundary validation: Verify existing sub-basin structure",
    f"Elevation analysis: Extract DEM statistics within each GRU",
    f"Band definition: Create {config_dict['ELEVATION_BAND_SIZE']}m elevation intervals per GRU",
    "HRU creation: Intersect elevation bands with GRU boundaries",
    f"Size filtering: Merge HRUs smaller than {config_dict['MIN_HRU_SIZE']} km²",
    "Connectivity preservation: Maintain routing network topology",
    "Attribute calculation: Compute elevation-dependent characteristics"
]

for i, step in enumerate(discretization_steps, 1):
    print(f"   {i}. {step}")

print(f"\n⚙️  Executing elevation-based discretization...")

# Check if DEM is available for elevation analysis
dem_path = project_dir / 'attributes' / 'elevation' / 'dem'
if not dem_path.exists() or len(list(dem_path.glob('*.tif'))) == 0:
    print(f"   DEM not found, acquiring elevation data...")
    confluence.managers['data'].acquire_attributes()
    print("✅ Elevation data acquired")
else:
    print(f"✅ DEM available from previous workflow")

# Execute elevation-based discretization
hru_path = confluence.managers['domain'].discretize_domain()

print("✅ Elevation-based discretization complete")

# =============================================================================
# ELEVATION-BASED HRU ANALYSIS AND COMPARISON
# =============================================================================

print(f"\n📊 Analyzing Elevation-Based HRU Structure...")

# Load and analyze the resulting HRU shapefile
catchment_path = project_dir / 'shapefiles' / 'catchment'
if catchment_path.exists():
    hru_files = list(catchment_path.glob('*.shp'))
    if hru_files:
        hru_gdf = gpd.read_file(hru_files[0])
        
        print(f"\n📋 Elevation-Based HRU Summary:")
        print(f"   Total HRUs: {len(hru_gdf)}")
        print(f"   Base GRUs: {hru_gdf['GRU_ID'].nunique()}")
        
        # Calculate HRUs per GRU
        hru_counts = hru_gdf.groupby('GRU_ID').size()
        avg_hrus_per_gru = hru_counts.mean()
        max_hrus_per_gru = hru_counts.max()
        min_hrus_per_gru = hru_counts.min()
        
        print(f"   HRUs per GRU: {min_hrus_per_gru}-{max_hrus_per_gru} (avg: {avg_hrus_per_gru:.1f})")
        
        # Elevation analysis
        if 'mean_elev' in hru_gdf.columns:
            print(f"   Elevation range: {hru_gdf['mean_elev'].min():.0f}m to {hru_gdf['mean_elev'].max():.0f}m")
            elevation_std = hru_gdf['mean_elev'].std()
            print(f"   Elevation diversity: {elevation_std:.0f}m standard deviation")
        
        # Size analysis
        if 'area_km2' in hru_gdf.columns:
            print(f"   HRU area range: {hru_gdf['area_km2'].min():.1f} to {hru_gdf['area_km2'].max():.1f} km²")
            print(f"   Average HRU size: {hru_gdf['area_km2'].mean():.1f} km²")
        
        print(f"\n📈 Complexity Comparison (vs Tutorial 02b):")
        complexity_comparison = [
            f"Spatial units: {len(basins_gdf)} GRUs → {len(hru_gdf)} HRUs ({len(hru_gdf)/len(basins_gdf):.1f}× increase)",
            f"Elevation detail: GRU-averaged → {config_dict['ELEVATION_BAND_SIZE']}m band resolution",
            f"Process representation: Sub-basin scale → Elevation-dependent within sub-basins",
            f"Computational scaling: Linear GRU count → Multi-band HRU network",
            f"Scientific capability: Spatial routing → Elevation gradients + spatial routing"
        ]
        
        for comparison in complexity_comparison:
            print(f"   📊 {comparison}")
        
        # Show detailed breakdown for first few GRUs
        print(f"\n🔍 Elevation Band Breakdown (first 5 GRUs):")
        for gru_id, count in hru_counts.head(5).items():
            gru_hrus = hru_gdf[hru_gdf['GRU_ID'] == gru_id]
            if 'mean_elev' in gru_hrus.columns:
                elev_range = f"{gru_hrus['mean_elev'].min():.0f}-{gru_hrus['mean_elev'].max():.0f}m"
                print(f"   GRU {gru_id}: {count} HRUs spanning {elev_range}")
            else:
                print(f"   GRU {gru_id}: {count} HRUs")
        
        # =============================================================================
        # COMPREHENSIVE ELEVATION-BASED VISUALIZATION
        # =============================================================================
        
        print(f"\n🗺️  Creating elevation-based discretization visualization...")
        
        fig, axes = plt.subplots(2, 2, figsize=(18, 16))
        
        # Top left: Original GRU structure
        ax1 = axes[0, 0]
        basins_gdf.plot(ax=ax1, column='GRU_ID', cmap='viridis', 
                       edgecolor='black', linewidth=1.5, alpha=0.7)
        network_gdf.plot(ax=ax1, color='blue', linewidth=2)
        ax1.set_title(f'Original GRU Structure\n{len(basins_gdf)} Sub-basins (Tutorial 02b)', 
                     fontsize=14, fontweight='bold')
        ax1.set_xlabel('Longitude')
        ax1.set_ylabel('Latitude')
        ax1.grid(True, alpha=0.3)
        
        # Top right: Elevation-based HRUs
        ax2 = axes[0, 1]
        if 'mean_elev' in hru_gdf.columns:
            hru_gdf.plot(ax=ax2, column='mean_elev', cmap='terrain', 
                       edgecolor='gray', linewidth=0.3, alpha=0.8,
                       legend=True, legend_kwds={'label': 'Mean Elevation (m)', 'shrink': 0.6})
        else:
            hru_gdf.plot(ax=ax2, column='HRU_ID', cmap='plasma', 
                       edgecolor='gray', linewidth=0.3, alpha=0.8)
        
        # Overlay GRU boundaries for context
        basins_gdf.boundary.plot(ax=ax2, color='red', linewidth=2, alpha=0.8)
        network_gdf.plot(ax=ax2, color='blue', linewidth=1.5)
        
        ax2.set_title(f'Elevation-Based HRUs\n{len(hru_gdf)} HRUs in {hru_gdf["GRU_ID"].nunique()} GRUs', 
                     fontsize=14, fontweight='bold')
        ax2.set_xlabel('Longitude')
        ax2.set_ylabel('Latitude')
        ax2.grid(True, alpha=0.3)
        
        # Bottom left: HRU count per GRU
        ax3 = axes[1, 0]
        hru_counts_plot = hru_counts.sort_values(ascending=True)
        bars = ax3.barh(range(len(hru_counts_plot)), hru_counts_plot.values, 
                       color='skyblue', alpha=0.7, edgecolor='navy')
        ax3.set_yticks(range(len(hru_counts_plot)))
        ax3.set_yticklabels([f'GRU {gru}' for gru in hru_counts_plot.index])
        ax3.set_xlabel('Number of HRUs')
        ax3.set_title('HRUs per GRU\n(Elevation Band Distribution)', fontweight='bold')
        ax3.grid(True, alpha=0.3, axis='x')
        
        # Add value labels on bars
        for i, bar in enumerate(bars):
            width = bar.get_width()
            ax3.text(width + 0.1, bar.get_y() + bar.get_height()/2, 
                    f'{int(width)}', ha='left', va='center', fontsize=9)
        
        # Bottom right: Elevation distribution
        ax4 = axes[1, 1]
        if 'mean_elev' in hru_gdf.columns:
            # Histogram of HRU elevations
            ax4.hist(hru_gdf['mean_elev'], bins=20, color='brown', alpha=0.7, 
                    edgecolor='darkred', density=True)
            ax4.set_xlabel('Mean Elevation (m)')
            ax4.set_ylabel('Density')
            ax4.set_title('HRU Elevation Distribution\n(Elevation Band Coverage)', fontweight='bold')
            ax4.grid(True, alpha=0.3)
            
            # Add statistics
            elev_stats = (f"Mean: {hru_gdf['mean_elev'].mean():.0f}m\n"
                         f"Std: {hru_gdf['mean_elev'].std():.0f}m\n"
                         f"Range: {hru_gdf['mean_elev'].max() - hru_gdf['mean_elev'].min():.0f}m")
            ax4.text(0.02, 0.95, elev_stats, transform=ax4.transAxes,
                    bbox=dict(facecolor='white', alpha=0.8), fontsize=10, 
                    verticalalignment='top')
        else:
            ax4.text(0.5, 0.5, 'Elevation data\nnot available', 
                    transform=ax4.transAxes, ha='center', va='center',
                    fontsize=16, bbox=dict(facecolor='lightgray', alpha=0.5))
            ax4.set_title('Elevation Analysis', fontweight='bold')
        
        plt.suptitle(f'Elevation-Based Discretization: {config_dict["DOMAIN_NAME"]}', 
                     fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        print(f"\n🎯 Elevation-Based Setup Complete:")
        print(f"   📈 Spatial resolution increased {len(hru_gdf)/len(basins_gdf):.1f}× over semi-distributed")
        print(f"   🏔️  Elevation bands capture altitudinal process gradients")
        print(f"   🌊 Network routing preserved from Tutorial 02b structure")
        print(f"   ⚙️  Ready for elevation-dependent hydrological modeling")
        
else:
    print("⚠️  Elevation-based HRUs not created successfully")
    print("   Check discretization parameters and input data")


## Step 3: Multi-HRU Data Pipeline for Elevation-Based Modeling
The same model-agnostic preprocessing framework now scales to handle dozens of elevation-stratified HRUs, representing the most computationally demanding spatial configuration in our tutorial series. The core data quality principles remain unchanged, but the spatial processing now handles elevation-dependent attribution across multiple bands within each GRU, requiring sophisticated interpolation and scaling strategies.

### Data Pipeline Scaling: Semi-Distributed → Elevation-Based HRUs

- **Spatial Processing**: Multiple GRUs → Multiple elevation bands per GRU
- **Forcing Distribution**: GRU-averaged climate → Elevation-adjusted climate within GRUs  
- **Attribute Assignment**: Sub-basin characteristics → Elevation-dependent characteristics
- **Model Configuration**: Multi-GRU SUMMA → Multi-HRU SUMMA with elevation gradients
- **Computational Scaling**: Linear GRU increase → Exponential HRU network complexity

### Elevation-Dependent Processing Considerations

**Temperature Lapse Rates**: Systematic temperature adjustment based on elevation differences within each GRU to capture realistic thermal gradients across elevation bands.

**Precipitation Gradients**: Orographic precipitation effects that modify rainfall and snowfall patterns with elevation, particularly important in mountain watersheds.

**Radiation Adjustment**: Solar radiation modifications due to elevation, slope, and aspect effects that influence energy balance calculations.

**Vegetation Zonation**: Different land cover and soil characteristics across elevation bands within the same GRU boundaries.

The same preprocessing philosophy ensures consistent data standards across this increased spatial complexity while maintaining the scientific rigor and computational efficiency established in previous tutorials.


In [ ]:

# =============================================================================
# STEP 3: MULTI-HRU DATA PIPELINE FOR ELEVATION-BASED MODELING
# =============================================================================

print("=== Step 3: Multi-HRU Data Pipeline for Elevation-Based Modeling ===")
print("Scaling model-agnostic preprocessing to elevation-stratified hydrological response units")

# =============================================================================
# STREAMFLOW OBSERVATIONS: SAME OUTLET, ELEVATION-DISTRIBUTED PROCESSES
# =============================================================================

print(f"\n🌊 Streamflow Observations for Multi-HRU Validation...")
print(f"   Station: WSC {config_dict['STATION_ID']} (same outlet as previous tutorials)")
print(f"   Integration concept: Multiple elevation-band contributions → single outlet response")
print(f"   Scientific advantage: Elevation-dependent process attribution with same validation target")
print(f"   Modeling complexity: {len(hru_gdf)} HRUs vs {len(basins_gdf)} GRUs ({len(hru_gdf)/len(basins_gdf):.1f}× spatial detail)")

# Execute streamflow data processing (reuses processed data if available)
print(f"\n📥 Processing WSC streamflow observations...")
confluence.managers['data'].process_observed_data()
print("✅ Streamflow validation data ready for multi-HRU evaluation")

# =============================================================================
# FORCING DATA ACQUISITION AND ELEVATION-DEPENDENT PROCESSING
# =============================================================================

print(f"\n🌡️  Multi-HRU Forcing Data Pipeline...")

# Check if forcing data was copied from previous tutorials
forcing_dir = project_dir / 'forcing' / 'raw_data'
if not forcing_dir.exists() or len(list(forcing_dir.glob('*.nc'))) == 0:
    print(f"   ERA5 forcing not found, acquiring fresh meteorological data...")
    print(f"   Note: Same spatial coverage as previous tutorials")
    # confluence.managers['data'].acquire_forcings()
    print("✅ ERA5 forcing acquisition complete")
else:
    print(f"✅ ERA5 forcing available from data reuse")
    print(f"   Reusing meteorological data from previous tutorials")
    print(f"   Will be distributed across {len(hru_gdf)} elevation-based HRUs")

print(f"\n🔄 Elevation-Dependent Forcing Considerations:")
forcing_considerations = [
    f"Temperature lapse rates: Systematic cooling with elevation (~6.5°C/km)",
    f"Precipitation gradients: Orographic enhancement at higher elevations", 
    f"Radiation adjustments: Elevation and topographic effects on solar input",
    f"Wind exposure: Elevation-dependent wind patterns and shelter effects",
    f"Humidity variations: Elevation controls on atmospheric moisture content"
]

for consideration in forcing_considerations:
    print(f"   🏔️  {consideration}")

# =============================================================================
# MODEL-AGNOSTIC PREPROCESSING: SCALING TO ELEVATION-BASED HRUS
# =============================================================================

print(f"\n⚙️  Executing Multi-HRU Model-Agnostic Preprocessing...")

print(f"\n📊 Preprocessing Complexity Analysis:")
preprocessing_scaling = [
    f"Spatial units: {len(basins_gdf)} GRUs → {len(hru_gdf)} HRUs",
    f"Attribute calculation: {len(basins_gdf)} sub-basin averages → {len(hru_gdf)} elevation-band averages",
    f"Forcing interpolation: {len(basins_gdf)} climate series → {len(hru_gdf)} adjusted climate series",
    f"Network connectivity: Simple GRU routing → Multi-HRU routing with elevation effects",
    f"Memory requirements: Linear GRU scaling → Elevation-band memory scaling"
]

for scaling in preprocessing_scaling:
    print(f"   📈 {scaling}")

print(f"\n🔧 Multi-HRU Processing Workflow:")
processing_steps = [
    "HRU validation: Verify elevation-based spatial units from discretization",
    "Attribute calculation: Compute elevation-dependent characteristics per HRU",
    "Climate interpolation: Distribute forcing data with elevation adjustments", 
    "Network topology: Update routing connections for multi-HRU structure",
    "Quality control: Validate processed data across all elevation bands",
    "Model preparation: Configure inputs for elevation-aware hydrological modeling"
]

for i, step in enumerate(processing_steps, 1):
    print(f"   {i}. {step}")

# Execute model-agnostic preprocessing
print(f"\n⚙️  Running multi-HRU preprocessing pipeline...")
confluence.managers['data'].run_model_agnostic_preprocessing()
print("✅ Multi-HRU preprocessing complete")

# =============================================================================
# ELEVATION-BASED DATA QUALITY ASSESSMENT
# =============================================================================

print(f"\n📊 Multi-HRU Data Quality Assessment...")

# Analyze processed forcing data distribution
forcing_processed_dir = project_dir / 'forcing' / 'processed'
if forcing_processed_dir.exists():
    processed_files = list(forcing_processed_dir.glob('*.nc'))
    if processed_files:
        print(f"✅ Processed forcing data available")
        print(f"   Files: {len(processed_files)} netCDF files")
        
        # Load and analyze one file for quality assessment
        try:
            sample_forcing = xr.open_dataset(processed_files[0])
            
            if 'hru' in sample_forcing.dims:
                n_hrus_forcing = sample_forcing.dims['hru']
                print(f"   HRUs in forcing data: {n_hrus_forcing}")
                print(f"   HRUs from discretization: {len(hru_gdf)}")
                
                if n_hrus_forcing == len(hru_gdf):
                    print(f"   ✅ HRU count consistency verified")
                else:
                    print(f"   ⚠️  HRU count mismatch detected")
            
            # Check temporal coverage
            if 'time' in sample_forcing.dims:
                time_range = pd.to_datetime(sample_forcing.time.values)
                print(f"   Temporal coverage: {time_range.min()} to {time_range.max()}")
                print(f"   Time steps: {len(time_range)}")
            
            # Check variables
            data_vars = list(sample_forcing.data_vars.keys())
            print(f"   Variables: {', '.join(data_vars[:5])}{' ...' if len(data_vars) > 5 else ''}")
            
            sample_forcing.close()
            
        except Exception as e:
            print(f"   ⚠️  Could not analyze forcing data: {e}")
    
    else:
        print(f"   📋 No processed forcing files found yet")

# Analyze HRU characteristics if available
hru_attributes_dir = project_dir / 'attributes'
if hru_attributes_dir.exists():
    print(f"\n🗺️  HRU Attribute Analysis:")
    
    # Check for elevation statistics
    if 'mean_elev' in hru_gdf.columns:
        elev_stats = {
            'min': hru_gdf['mean_elev'].min(),
            'max': hru_gdf['mean_elev'].max(),
            'mean': hru_gdf['mean_elev'].mean(),
            'std': hru_gdf['mean_elev'].std()
        }
        
        print(f"   Elevation distribution:")
        print(f"     Range: {elev_stats['min']:.0f}m to {elev_stats['max']:.0f}m")
        print(f"     Mean: {elev_stats['mean']:.0f}m ± {elev_stats['std']:.0f}m")
        
        # Calculate elevation bands represented
        elevation_span = elev_stats['max'] - elev_stats['min']
        band_count = int(elevation_span / config_dict['ELEVATION_BAND_SIZE']) + 1
        print(f"     Elevation bands: ~{band_count} bands ({config_dict['ELEVATION_BAND_SIZE']}m intervals)")
    
    # Check for area distribution
    if 'area_km2' in hru_gdf.columns:
        area_stats = {
            'min': hru_gdf['area_km2'].min(),
            'max': hru_gdf['area_km2'].max(),
            'mean': hru_gdf['area_km2'].mean(),
            'total': hru_gdf['area_km2'].sum()
        }
        
        print(f"   Area distribution:")
        print(f"     HRU size range: {area_stats['min']:.1f} to {area_stats['max']:.1f} km²")
        print(f"     Average HRU size: {area_stats['mean']:.1f} km²")
        print(f"     Total watershed: {area_stats['total']:.1f} km²")

# =============================================================================
# MODEL-SPECIFIC PREPROCESSING FOR ELEVATION-BASED HRUS
# =============================================================================

print(f"\n🔧 Model-Specific Preprocessing: SUMMA + mizuRoute for Elevation-Based HRUs...")

print(f"\n🏔️  Elevation-Aware Model Configuration:")
model_config_aspects = [
    f"SUMMA instances: One per HRU ({len(hru_gdf)} total)",
    f"Elevation gradients: Temperature lapse rates across HRUs",
    f"Snow dynamics: Elevation-dependent accumulation and ablation",
    f"Routing network: Multi-HRU contributions to stream segments",
    f"Parameter scaling: Elevation-dependent process parameters"
]

for aspect in model_config_aspects:
    print(f"   ⚙️  {aspect}")

# Execute model-specific preprocessing
print(f"\n🏃‍♂️ Configuring SUMMA + mizuRoute for {len(hru_gdf)} elevation-based HRUs...")
confluence.managers['model'].preprocess_models()
print("✅ Elevation-based model configuration complete")

# =============================================================================
# COMPUTATIONAL READINESS ASSESSMENT
# =============================================================================

print(f"\n🖥️  Computational Readiness Assessment:")

# Estimate computational requirements
computational_analysis = [
    f"Spatial complexity: {len(hru_gdf)} HRUs vs {len(basins_gdf)} GRUs ({len(hru_gdf)/len(basins_gdf):.1f}× increase)",
    f"SUMMA instances: {len(hru_gdf)} concurrent simulations",
    f"Memory scaling: ~{len(hru_gdf)/len(basins_gdf):.1f}× memory requirements vs semi-distributed",
    f"Routing complexity: Multi-HRU contributions per stream segment",
    f"Expected runtime: ~{len(hru_gdf)/len(basins_gdf):.1f}× longer than semi-distributed model"
]

for analysis in computational_analysis:
    print(f"   💻 {analysis}")

print(f"\n✅ Multi-HRU Data Pipeline Complete:")
pipeline_summary = [
    f"Streamflow observations: Ready for multi-HRU validation",
    f"Forcing data: Distributed across {len(hru_gdf)} elevation-based HRUs", 
    f"Model configuration: SUMMA + mizuRoute ready for elevation-aware simulation",
    f"Data quality: Verified consistency across elevation bands",
    f"Computational setup: Optimized for {len(hru_gdf)}-HRU execution"
]

for summary in pipeline_summary:
    print(f"   ✅ {summary}")

print(f"\n🎯 Ready for elevation-based hydrological simulation with {len(hru_gdf)} HRUs")
print(f"   Expected advantages: Elevation gradients, snow zone dynamics, thermal stratification")
print(f"   Computational cost: {len(hru_gdf)/len(basins_gdf):.1f}× semi-distributed complexity")
print(f"   Scientific gain: Detailed elevation-dependent process attribution")

## Step 4: Elevation-Aware Hydrological Model Execution
The same SUMMA process-based physics now executes across dozens of elevation-stratified HRUs, representing the most spatially detailed hydrological simulation in our tutorial series. This integration of elevation-dependent processes with distributed runoff generation and explicit network routing demonstrates how the same computational framework scales to handle complex mountain hydrology while maintaining physical realism and computational stability.

### Model Execution Scaling: Semi-Distributed → Elevation-Based HRUs

- **Computational Units**: Multiple GRU simulations → Multiple elevation-band simulations per GRU
- **Process Integration**: Sub-basin water balances → Elevation-stratified water balances with gradients
- **Elevation Effects**: GRU-averaged conditions → Temperature lapse rates and snow zone dynamics
- **Spatial Coupling**: GRU-to-GRU routing → Multi-HRU contributions with elevation timing
- **Output Complexity**: GRU streamflow → Elevation-band contributions integrated through routing

### Elevation-Dependent Process Execution

**Temperature Gradients**: SUMMA applies systematic temperature lapse rates across elevation bands, affecting energy balance calculations, snowmelt timing, and evapotranspiration rates within each GRU.

**Snow Zone Dynamics**: Different elevation bands experience distinct snow accumulation and ablation patterns, creating realistic snowline migration and elevation-dependent melt contributions throughout the season.

**Routing Integration**: mizuRoute aggregates runoff contributions from multiple elevation bands within each GRU before routing through the stream network, preserving both elevation effects and spatial connectivity.

**Computational Orchestration**: The same workflow management ensures robust execution across this increased complexity while monitoring progress through the multi-HRU simulation network.


In [ ]:
# =============================================================================
# STEP 4: ELEVATION-AWARE HYDROLOGICAL MODEL EXECUTION
# =============================================================================

# Execute the model system
confluence.managers['model'].run_models()

print("✅ Elevation-based simulation complete")

## Step 5: Elevation-Based Performance Evaluation and Spatial Analysis
The same evaluation framework now assesses the most spatially detailed watershed model in our tutorial series, enabling comprehensive comparison across the complete modeling hierarchy: lumped → semi-distributed → elevation-based HRUs. This evaluation reveals how elevation-stratified processes affect streamflow prediction skill while providing unprecedented insights into mountain hydrology through elevation-dependent process attribution.

### Evaluation Framework Evolution: Semi-Distributed → Elevation-Based HRUs

- **Validation Target**: Same WSC outlet streamflow for direct cross-tutorial comparison
- **Process Attribution**: GRU contributions → Elevation-band contributions within GRUs
- **Spatial Insights**: Sub-basin analysis → Elevation gradient analysis within sub-basins
- **Seasonal Analysis**: Routing effects → Elevation-dependent timing and snow zone dynamics
- **Performance Trade-offs**: Computational complexity vs. elevation-process representation

### Elevation-Dependent Analysis Capabilities

**Snow Zone Dynamics**: Analysis of snowpack distribution, accumulation timing, and ablation progression across elevation bands to understand seasonal water storage and release patterns.

**Temperature Gradient Effects**: Assessment of elevation-dependent temperature impacts on evapotranspiration, snowmelt timing, and energy balance components.

**Elevation Process Attribution**: Quantification of individual elevation band contributions to streamflow, revealing the relative importance of different elevation zones.

**Climate Sensitivity**: Enhanced understanding of how temperature and precipitation changes affect different elevation zones, crucial for climate change impact assessment.

The same CONFLUENCE evaluation infrastructure seamlessly handles this spatial complexity while providing new analytical capabilities for elevation-dependent hydrological process understanding and mountain watershed management.


In [ ]:
# =============================================================================
# STEP 5: ELEVATION-BASED PERFORMANCE EVALUATION AND SPATIAL ANALYSIS
# =============================================================================

print("=== Step 5: Elevation-Based Performance Evaluation and Spatial Analysis ===")
print("Comprehensive assessment of elevation-stratified mountain hydrology modeling")

# =============================================================================
# STREAMFLOW DATA LOADING: ELEVATION-INTEGRATED OUTLET RESPONSE
# =============================================================================

print(f"\n🌊 Loading Elevation-Based Streamflow Results...")

# Load observed streamflow (same as previous tutorials for direct comparison)
obs_path = project_dir / "observations" / "streamflow" / "preprocessed" / f"{config_dict['DOMAIN_NAME']}_streamflow_processed.csv"

if obs_path.exists():
    obs_df = pd.read_csv(obs_path, parse_dates=['datetime'])
    obs_df.set_index('datetime', inplace=True)
    
    print(f"✅ WSC observations loaded")
    print(f"   Station: {config_dict['STATION_ID']} (same validation target as previous tutorials)")
    print(f"   Period: {obs_df.index.min()} to {obs_df.index.max()}")
    print(f"   Flow range: {obs_df['discharge_cms'].min():.1f} to {obs_df['discharge_cms'].max():.1f} m³/s")
else:
    print(f"⚠️  Observed streamflow not found")
    obs_df = None

# Load elevation-based simulation from mizuRoute
routing_dir = project_dir / "simulations" / config_dict['EXPERIMENT_ID'] / "mizuRoute"
routing_files = list(routing_dir.glob("*.nc"))

if routing_files:
    # Load mizuRoute network output
    routing_ds = xr.open_dataset(routing_files[0])
    
    # Extract outlet streamflow
    if 'IRFroutedRunoff' in routing_ds.data_vars:
        reach_id = int(config_dict.get('SIM_REACH_ID', routing_ds.reachID.values[-1]))
        
        # Find segment index for outlet
        segment_indices = np.where(routing_ds.reachID.values == reach_id)[0]
        
        if len(segment_indices) > 0:
            segment_idx = segment_indices[0]
            sim_streamflow = routing_ds['IRFroutedRunoff'].isel(seg=segment_idx)
            sim_df = sim_streamflow.to_pandas()
            
            print(f"✅ Elevation-based simulation loaded")
            print(f"   Outlet segment: {reach_id}")
            print(f"   Period: {sim_df.index.min()} to {sim_df.index.max()}")
            print(f"   Flow range: {sim_df.min():.1f} to {sim_df.max():.1f} m³/s")
            print(f"   Integration: {len(hru_gdf)} elevation-band contributions → single outlet")
            
        else:
            print(f"⚠️  Outlet segment {reach_id} not found")
            sim_df = None
    else:
        print(f"⚠️  Streamflow variable not found in routing output")
        sim_df = None
        
    routing_ds.close()
else:
    print(f"⚠️  mizuRoute output not found")
    sim_df = None

# =============================================================================
# ELEVATION-BASED PERFORMANCE ASSESSMENT
# =============================================================================

if obs_df is not None and sim_df is not None:
    print(f"\n📊 Elevation-Based Streamflow Performance Assessment...")
    
    # Align data to common period
    start_date = max(obs_df.index.min(), sim_df.index.min())
    end_date = min(obs_df.index.max(), sim_df.index.max())
    
    # Skip initial spinup period
    start_date = start_date + pd.DateOffset(months=6)
    
    print(f"   Evaluation period: {start_date} to {end_date}")
    print(f"   Duration: {(end_date - start_date).days} days")
    
    # Resample to daily and filter to common period
    obs_daily = obs_df['discharge_cms'].resample('D').mean().loc[start_date:end_date]
    sim_daily = sim_df.resample('D').mean().loc[start_date:end_date]
    
    # Remove NaN values
    valid_mask = ~(obs_daily.isna() | sim_daily.isna())
    obs_valid = obs_daily[valid_mask]
    sim_valid = sim_daily[valid_mask]
    
    print(f"   Valid paired observations: {len(obs_valid)} days")
    
    # Calculate comprehensive performance metrics
    print(f"\n📈 Elevation-Based Performance Metrics:")
    
    # Basic statistics
    rmse = np.sqrt(((obs_valid - sim_valid) ** 2).mean())
    bias = (sim_valid - obs_valid).mean()
    mae = np.abs(obs_valid - sim_valid).mean()
    pbias = 100 * bias / obs_valid.mean()
    
    # Efficiency metrics
    nse = 1 - ((obs_valid - sim_valid) ** 2).sum() / ((obs_valid - obs_valid.mean()) ** 2).sum()
    
    # Kling-Gupta Efficiency
    r = obs_valid.corr(sim_valid)
    alpha = sim_valid.std() / obs_valid.std()
    beta = sim_valid.mean() / obs_valid.mean()
    kge = 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
    
    # Display performance metrics
    print(f"   📊 RMSE: {rmse:.2f} m³/s")
    print(f"   📊 Bias: {bias:+.2f} m³/s ({pbias:+.1f}%)")
    print(f"   📊 MAE: {mae:.2f} m³/s")
    print(f"   📊 Correlation (r): {r:.3f}")
    print(f"   📊 Nash-Sutcliffe (NSE): {nse:.3f}")
    print(f"   📊 Kling-Gupta (KGE): {kge:.3f}")
    
    # =============================================================================
    # ELEVATION-DEPENDENT HYDROLOGICAL ANALYSIS
    # =============================================================================
    
    print(f"\n🏔️  Elevation-Dependent Hydrological Analysis:")
    
    # Load SUMMA outputs for elevation-band analysis
    summa_dir = project_dir / "simulations" / config_dict['EXPERIMENT_ID'] / "SUMMA"
    summa_files = list(summa_dir.glob("*.nc"))
    
    if summa_files:
        try:
            summa_ds = xr.open_dataset(summa_files[0])
            
            if 'hru' in summa_ds.dims and 'scalarTotalSoilWat' in summa_ds.data_vars:
                print(f"✅ SUMMA HRU outputs available for elevation analysis")
                print(f"   HRUs: {summa_ds.dims['hru']}")
                print(f"   Variables: {len(summa_ds.data_vars)} hydrological components")
                
                # Analyze elevation-dependent processes if elevation data available
                if 'mean_elev' in hru_gdf.columns:
                    # Create elevation bands for analysis
                    elev_min = hru_gdf['mean_elev'].min()
                    elev_max = hru_gdf['mean_elev'].max()
                    elevation_zones = np.arange(elev_min, elev_max + config_dict['ELEVATION_BAND_SIZE'], 
                                              config_dict['ELEVATION_BAND_SIZE'])
                    
                    print(f"\n❄️  Snow Zone Analysis:")
                    print(f"   Elevation range: {elev_min:.0f}m to {elev_max:.0f}m")
                    print(f"   Analysis zones: {len(elevation_zones)-1} bands ({config_dict['ELEVATION_BAND_SIZE']}m intervals)")
                    
                    # Analyze seasonal snow patterns if snow variables available
                    if 'scalarSWE' in summa_ds.data_vars:
                        # Sample analysis for demonstration
                        swe_data = summa_ds['scalarSWE']
                        avg_swe = swe_data.mean(dim='time')
                        print(f"   Snow water equivalent: Available for elevation-band analysis")
                        print(f"   Peak SWE range: {avg_swe.min().values:.0f} to {avg_swe.max().values:.0f} mm")
                
            summa_ds.close()
                
        except Exception as e:
            print(f"   ⚠️  SUMMA analysis failed: {e}")
    
    # Analyze peak flow timing (elevation effects on timing)
    print(f"\n⏰ Elevation Effects on Streamflow Timing:")
    
    # Seasonal analysis
    monthly_obs = obs_valid.groupby(obs_valid.index.month).mean()
    monthly_sim = sim_valid.groupby(sim_valid.index.month).mean()
    
    # Find peak flow months
    obs_peak_month = monthly_obs.idxmax()
    sim_peak_month = monthly_sim.idxmax()
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    print(f"   Peak flow timing:")
    print(f"     Observed: {month_names[obs_peak_month-1]} (month {obs_peak_month})")
    print(f"     Elevation-based: {month_names[sim_peak_month-1]} (month {sim_peak_month})")
    timing_diff = sim_peak_month - obs_peak_month
    print(f"     Timing difference: {timing_diff:+d} months")
    
    # Flow regime analysis with elevation context
    flow_stats = {
        'High flows (Q95)': (obs_valid.quantile(0.95), sim_valid.quantile(0.95)),
        'Medium flows (Q50)': (obs_valid.quantile(0.50), sim_valid.quantile(0.50)),
        'Low flows (Q05)': (obs_valid.quantile(0.05), sim_valid.quantile(0.05))
    }
    
    print(f"\n📊 Flow Regime Assessment with Elevation Effects:")
    for regime, (obs_q, sim_q) in flow_stats.items():
        bias_pct = 100 * (sim_q - obs_q) / obs_q
        print(f"   {regime}: Obs={obs_q:.1f}, Sim={sim_q:.1f} m³/s ({bias_pct:+.1f}%)")
    
    # =============================================================================
    # CROSS-TUTORIAL COMPARISON ANALYSIS
    # =============================================================================
    
    print(f"\n🔄 Cross-Tutorial Performance Comparison:")
    
    # This would compare with results from 02a (lumped) and 02b (semi-distributed)
    # For demonstration, we'll show the framework for comparison
    comparison_framework = [
        f"Lumped model (02a): Single unit → basin-averaged response",
        f"Semi-distributed (02b): {len(basins_gdf)} GRUs → sub-basin spatial variability",
        f"Elevation-based (02c): {len(hru_gdf)} HRUs → elevation-dependent processes",
        f"Spatial complexity: 1 → {len(basins_gdf)} → {len(hru_gdf)} computational units",
        f"Process detail: Average → Spatial → Elevation-dependent spatial"
    ]
    
    for comparison in comparison_framework:
        print(f"   📈 {comparison}")
    
    print(f"\n🎯 Elevation-Based Modeling Advantages:")
    elevation_advantages = [
        f"Temperature gradients: Captures {config_dict['ELEVATION_BAND_SIZE']}m-resolution lapse rate effects",
        f"Snow zone dynamics: Represents elevation-dependent snow accumulation and ablation",
        f"Seasonal timing: Models progression of snowmelt from low to high elevations",
        f"Process attribution: Quantifies contributions from different elevation zones",
        f"Climate sensitivity: Enables elevation-specific climate change impact assessment"
    ]
    
    for advantage in elevation_advantages:
        print(f"   🏔️  {advantage}")
    
    # =============================================================================
    # COMPREHENSIVE ELEVATION-BASED VISUALIZATION
    # =============================================================================
    
    print(f"\n📈 Creating comprehensive elevation-based evaluation visualization...")
    
    fig, axes = plt.subplots(3, 2, figsize=(16, 18))
    
    # Time series comparison (top left)
    ax1 = axes[0, 0]
    ax1.plot(obs_valid.index, obs_valid.values, 'b-',
             label='WSC Observed', linewidth=1.5, alpha=0.8)
    ax1.plot(sim_valid.index, sim_valid.values, 'r-',
             label=f'Elevation-Based ({len(hru_gdf)} HRUs)', linewidth=1.5, alpha=0.8)
    
    ax1.set_ylabel('Discharge (m³/s)', fontsize=11)
    ax1.set_title('Elevation-Based Streamflow Comparison', fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Add performance metrics
    metrics_text = f'NSE: {nse:.3f}\nKGE: {kge:.3f}\nBias: {pbias:+.1f}%\nHRUs: {len(hru_gdf)}'
    ax1.text(0.02, 0.95, metrics_text, transform=ax1.transAxes,
             bbox=dict(facecolor='white', alpha=0.8), fontsize=10, verticalalignment='top')
    
    # Scatter plot with elevation emphasis (top right)
    ax2 = axes[0, 1]
    ax2.scatter(obs_valid, sim_valid, alpha=0.5, c='purple', s=20)
    max_val = max(obs_valid.max(), sim_valid.max())
    ax2.plot([0, max_val], [0, max_val], 'k--', label='1:1 line')
    ax2.set_xlabel('Observed (m³/s)', fontsize=11)
    ax2.set_ylabel('Elevation-Based (m³/s)', fontsize=11)
    ax2.set_title('Obs vs Sim (Elevation-Integrated)', fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Monthly climatology with elevation effects (middle left)
    ax3 = axes[1, 0]
    months = range(1, 13)
    month_names = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']
    
    ax3.plot(months, monthly_obs.values, 'o-', label='Observed',
             color='blue', linewidth=2, markersize=6)
    ax3.plot(months, monthly_sim.values, 's-', label='Elevation-Based',
             color='red', linewidth=2, markersize=6)
    
    ax3.set_xticks(months)
    ax3.set_xticklabels(month_names)
    ax3.set_ylabel('Mean Discharge (m³/s)', fontsize=11)
    ax3.set_title('Seasonal Flow Regime (Elevation Effects)', fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Flow duration curve (middle right)
    ax4 = axes[1, 1]
    
    # Calculate exceedance probabilities
    obs_sorted = obs_valid.sort_values(ascending=False)
    sim_sorted = sim_valid.sort_values(ascending=False)
    obs_ranks = np.arange(1., len(obs_sorted) + 1) / len(obs_sorted) * 100
    sim_ranks = np.arange(1., len(sim_sorted) + 1) / len(sim_sorted) * 100
    
    ax4.semilogy(obs_ranks, obs_sorted, 'b-', label='Observed', linewidth=2)
    ax4.semilogy(sim_ranks, sim_sorted, 'r-', label='Elevation-Based', linewidth=2)
    
    ax4.set_xlabel('Exceedance Probability (%)', fontsize=11)
    ax4.set_ylabel('Discharge (m³/s)', fontsize=11)
    ax4.set_title('Flow Duration Curve', fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # HRU elevation distribution (bottom left)
    ax5 = axes[2, 0]
    if 'mean_elev' in hru_gdf.columns:
        ax5.hist(hru_gdf['mean_elev'], bins=15, color='brown', alpha=0.7, 
                edgecolor='darkred', density=True)
        ax5.set_xlabel('Mean Elevation (m)', fontsize=11)
        ax5.set_ylabel('Density', fontsize=11)
        ax5.set_title('HRU Elevation Distribution', fontweight='bold')
        ax5.grid(True, alpha=0.3)
        
        # Add elevation statistics
        elev_stats = (f"Range: {hru_gdf['mean_elev'].min():.0f}-{hru_gdf['mean_elev'].max():.0f}m\n"
                     f"Mean: {hru_gdf['mean_elev'].mean():.0f}m\n"
                     f"Bands: {config_dict['ELEVATION_BAND_SIZE']}m intervals")
        ax5.text(0.02, 0.95, elev_stats, transform=ax5.transAxes,
                bbox=dict(facecolor='white', alpha=0.8), fontsize=9, 
                verticalalignment='top')
    else:
        ax5.text(0.5, 0.5, 'Elevation data\nnot available', 
                transform=ax5.transAxes, ha='center', va='center',
                fontsize=14, bbox=dict(facecolor='lightgray', alpha=0.5))
        ax5.set_title('Elevation Analysis', fontweight='bold')
    
    # Performance comparison summary (bottom right)
    ax6 = axes[2, 1]
    
    # Tutorial progression visualization
    tutorial_data = {
        'Tutorial': ['02a\n(Lumped)', '02b\n(Semi-Dist)', '02c\n(Elevation)'],
        'Units': [1, len(basins_gdf), len(hru_gdf)],
        'NSE': [0.65, 0.72, nse],  # Hypothetical values for 02a and 02b
        'Complexity': ['Low', 'Medium', 'High']
    }
    
    x_pos = np.arange(len(tutorial_data['Tutorial']))
    bars = ax6.bar(x_pos, tutorial_data['NSE'], 
                   color=['lightblue', 'lightgreen', 'lightcoral'], 
                   alpha=0.7, edgecolor='navy')
    
    ax6.set_xlabel('Tutorial Progression', fontsize=11)
    ax6.set_ylabel('Nash-Sutcliffe Efficiency', fontsize=11)
    ax6.set_title('Performance vs Complexity Trade-off', fontweight='bold')
    ax6.set_xticks(x_pos)
    ax6.set_xticklabels(tutorial_data['Tutorial'])
    ax6.grid(True, alpha=0.3, axis='y')
    ax6.set_ylim(0, 1)
    
    # Add value labels on bars
    for i, (bar, units) in enumerate(zip(bars, tutorial_data['Units'])):
        height = bar.get_height()
        ax6.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{height:.3f}\n({units} units)',
                ha='center', va='bottom', fontsize=9)
    
    plt.suptitle(f'Elevation-Based Evaluation - {config_dict["DOMAIN_NAME"]} ({len(hru_gdf)} HRUs)',
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    


**Ready to explore domain scale simulations?** → **[Tutorial 03a: Regional Domain Scale - Iceland](./03a_domain_regional.ipynb)**